In [2]:
# Algoritmo de Búsqueda Lineal de Newton con Modificación a la Hessiana.

    # Algoritmos 3.1 y 3.2 de Nocedal

import numpy as np
print(np.__version__)

1.19.2


In [3]:
def gradiente(func,xk):
    # Código que calcula el gradiente de la función f en el punto xk
        # Aquí estamos suponiendo que metemos un xk array y lo convertimos a matriz de nx1
    cambio = .000001
    n = len(xk)
    grad = np.matrix(np.zeros((n,1)))
    
    for i in range(1,n+1):
        aux = xk.copy()
        aux[i-1,0] = aux[i-1,0] + cambio
        grad[i-1,0] = round((func(aux) - func(xk))/cambio,2)
    
    return grad

In [4]:
def hessiana(func,xk):
    # Código que calcula la hessiana de la función f en el punto xk
        # Aquí estamos suponiendo que metemos un xk matrix de nx1
    cambio = .000001
    n = len(xk)
    hess = np.matrix(np.zeros((n,n)))
    
    for i in range(1,n+1):
        for j in range(1,n+1):
            if i==j:
                aux1 = xk.copy()
                aux2 = xk.copy()
                aux1[i-1,0] = aux1[i-1,0] + cambio
                aux2[i-1,0] = aux2[i-1,0] - cambio
                hess[i-1,j-1] = round((func(aux1)-2*func(xk)+func(aux2))/cambio**2,2)
            else:
                aux1 = xk.copy()
                aux2 = xk.copy()
                aux3 = xk.copy()
                
                aux1[i-1,0] = aux1[i-1,0] + cambio
                aux1[j-1,0] = aux1[j-1,0] + cambio
                
                aux2[j-1,0] = aux2[j-1,0] + cambio
                
                aux3[i-1,0] = aux3[i-1,0] + cambio
                
                hess[i-1,j-1] = round((func(aux1)-func(aux2)-func(aux3)+func(xk))/cambio**2,2)
                
    return hess

In [5]:
# Función de Prueba de los Algoritmos (Rosenbrock)
def Rosenbrock(x):
    # Aquí suponemos que x ya es una matriz (vector) de nx1
    return (1-x[0,0])**2 + 100*(x[1,0]-x[0,0]**2)**2

In [6]:
# Algoritmo 3.1: Backtracking Line Search

def BLS(func,xk):
    a = 1 #Pues es la alpha que corresponde a la dirección de Newton
    c = 10**(-4)
    pk = -np.linalg.inv(hessiana(func,xk))*gradiente(func,xk)
    while func(xk+a*pk) > (func(xk) + c*a*np.transpose(gradiente(func,xk))*pk):
        a = .8*a
    return a

In [80]:
def posdef(func,xk):
    eigen = np.linalg.eig(hessiana(func,xk))[0]
    if all(eigen > 0):
        r = 1 #cumple hessiana positiva definida
    else:
        r = 0 #no cumple hessiana positiva definida
        
    return r

In [59]:
def menor_eigen(func,xk):
    #método que devuelve el menor de los eigenvalores de una matriz multiplicado por -1
    eigen = np.linalg.eig(hessiana(func,xk))[0]
    n = len(xk)
    min = 0
    for i in range(1,n+1):
        if eigen[i-1] < min:
            min = eigen[i-1]
    min = -min
    return min

In [88]:
# Algoritmo 3.2: Line Search Newton with Modification

def LSNM(func,x0):
    xk =  x0
    n = len(xk)
    for k in range(1,100):
        if posdef(func,xk)==1:
            Bk = hessiana(func,xk)
            pk = np.linalg.solve(Bk,-gradiente(func,xk)) 
            xk = xk + BLS(func,xk)*pk
        #else:
            # Bk = hessiana(func,xk) + menor_eigen(func,xk) * np.identity(n)
            # Aquí con la función menor_eigen
            # ya estamos considerando el caso en el que la matriz no es definida positiva y le estamos sumando
            # la identidad multiplicada por un factor (el negativo del menor de los eigenvalores), de esta forma
            # la obligamos a ser positiva definida.
            #pk = np.linalg.solve(Bk,-gradiente(func,xk)) 
            #xk = xk + BLS(func,xk)*pk
            
    return xk

In [92]:
x0 = np.matrix([[2.0],[1.0]])
LSNM(Rosenbrock,x0)

matrix([[0.99887989],
        [0.99773576]])